<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Deepfake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Deepfake and Manipulated Media Analysis Data Download**

In [1]:
!pip install -qU yt-dlp aiohttp decord pandas pillow soundfile tqdm crawl4ai
!pip install -qU crawl4ai[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.9/182.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/

In [4]:
from crawl4ai import AsyncWebCrawler
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy

In [5]:
import os
import asyncio
import aiohttp
from tqdm.asyncio import tqdm
import yt_dlp
from playwright.async_api import async_playwright
import pandas as pd

In [6]:
#########################################
# Setup Directories
#########################################
os.makedirs('scraped_urls', exist_ok=True)
os.makedirs('deepfake_images', exist_ok=True)
os.makedirs('deepfake_videos', exist_ok=True)
os.makedirs('deepfake_audios', exist_ok=True)

In [7]:
#########################################
# 1. Use Crawl4ai to Scrape Media URLs
#########################################
# Define source pages for each modality (example URLs - adjust as needed)
pages = {
    "images": "https://deepfakesampleimages.com/gallery",     # Example: a gallery page of deepfake images
    "videos": "https://deepfakesamplevideos.com/collection",    # Example: a collection page of deepfake videos
    "audio":  "https://deepfakesampleaudio.com/samples"          # Example: a page listing deepfake audio samples
}

In [8]:
# Define parse functions using CSS selectors (adjust selectors based on actual page structure)
def parse_image_links(response):
    # Extract image URLs from <img class="deepfake-img" src="...">
    return response.css("img.deepfake-img::attr(src)").getall()

def parse_video_links(response):
    # Extract video URLs from <a class="deepfake-video" href="...">
    return response.css("a.deepfake-video::attr(href)").getall()

def parse_audio_links(response):
    # Extract audio URLs from <audio class="deepfake-audio" src="...">
    return response.css("audio.deepfake-audio::attr(src)").getall()

# Create Crawl4ai tasks for each modality
tasks = [
    Task(name="image_urls", start_urls=[pages["images"]], parse_function=parse_image_links),
    Task(name="video_urls", start_urls=[pages["videos"]], parse_function=parse_video_links),
    Task(name="audio_urls", start_urls=[pages["audio"]], parse_function=parse_audio_links)
]

print("Starting Crawl4ai to scrape media URLs...")
crawler = Crawler(tasks=tasks, output_dir="scraped_urls")
crawler.run()
print("Crawl4ai scraping complete.")

NameError: name 'Task' is not defined

In [9]:
# Function to load scraped URLs from CSV files created by Crawl4ai.
def load_scraped_urls(modality):
    filepath = os.path.join("scraped_urls", f"{modality}_urls.csv")
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        # Assuming the CSV has a column 'url'
        return df['url'].tolist()
    else:
        print(f"File {filepath} not found. Using fallback method.")
        return []

# Load URLs for each modality
image_urls = load_scraped_urls("image")
video_urls = load_scraped_urls("video")
audio_urls = load_scraped_urls("audio")

# Fallbacks if no URLs were scraped:
if not image_urls:
    # For images, repeatedly use ThisPersonDoesNotExist which returns a new image each call.
    image_urls = ["https://thispersondoesnotexist.com/image"] * 20
else:
    image_urls = image_urls[:20]

if not video_urls:
    # For videos, we'll later use yt-dlp's search fallback.
    video_urls = []
if not audio_urls:
    # For audio, we will later extract using Playwright.
    audio_urls = []

File scraped_urls/image_urls.csv not found. Using fallback method.
File scraped_urls/video_urls.csv not found. Using fallback method.
File scraped_urls/audio_urls.csv not found. Using fallback method.


In [10]:
#########################################
# 2. Download Images Asynchronously using aiohttp and tqdm
#########################################
async def download_image(session, url, filename):
    try:
        async with session.get(url) as resp:
            if resp.status == 200:
                content = await resp.read()
                with open(filename, "wb") as f:
                    f.write(content)
                print(f"Downloaded image: {filename}")
            else:
                print(f"Failed to download image {url}: Status {resp.status}")
    except Exception as e:
        print(f"Exception while downloading image {url}: {e}")

async def download_images(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for idx, url in enumerate(urls):
            filename = os.path.join("deepfake_images", f"image_{idx+1:02d}.jpg")
            tasks.append(download_image(session, url, filename))
        await asyncio.gather(*tasks)


In [11]:
#########################################
# 3. Download Videos Using yt-dlp (Synchronous)
#########################################
def download_videos(num_videos=20):
    if video_urls:
        # Download from scraped video URLs
        download_list = video_urls[:num_videos]
        print("Downloading videos from scraped URLs...")
        for url in download_list:
            ydl_opts = {
                'format': 'bestvideo+bestaudio/best',
                'outtmpl': 'deepfake_videos/%(id)s.%(ext)s',
                'merge_output_format': 'mp4',
                'quiet': False,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                try:
                    ydl.download([url])
                except Exception as e:
                    print(f"Error downloading video {url}: {e}")
    else:
        # Fallback: use yt-dlp search to download 20 videos
        search_query = f"ytsearch20:deepfake compilation"
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',
            'outtmpl': 'deepfake_videos/%(id)s.%(ext)s',
            'merge_output_format': 'mp4',
            'quiet': False,
        }
        print("Downloading videos using yt-dlp search fallback...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([search_query])
    print("Video downloads complete.")

In [12]:
#########################################
# 4. Download Audios Asynchronously using Playwright & aiohttp
#########################################
async def extract_audio_links():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()
        url = "https://uberduck.ai/explore"
        await page.goto(url)
        await page.wait_for_timeout(5000)  # wait for dynamic content to load

        # Adjust selector according to actual structure – here we assume <audio class="sample-audio">
        audio_elements = await page.query_selector_all("audio.sample-audio")
        links = []
        for elem in audio_elements:
            src = await elem.get_attribute("src")
            if src:
                links.append(src)
        await browser.close()
        print(f"Extracted {len(links)} audio links from UberDuck.")
        return links[:20]

async def download_audio(session, url, filename):
    try:
        async with session.get(url) as resp:
            if resp.status == 200:
                content = await resp.read()
                with open(filename, "wb") as f:
                    f.write(content)
                print(f"Downloaded audio: {filename}")
            else:
                print(f"Failed to download audio {url}: Status {resp.status}")
    except Exception as e:
        print(f"Exception while downloading audio {url}: {e}")

async def download_audios(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for idx, url in enumerate(urls):
            filename = os.path.join("deepfake_audios", f"audio_{idx+1:02d}.mp3")
            tasks.append(download_audio(session, url, filename))
        await asyncio.gather(*tasks)

In [13]:
#########################################
# 5. Main Function to Run Entire Pipeline
#########################################
def main():
    loop = asyncio.get_event_loop()

    print("Starting image downloads...")
    loop.run_until_complete(download_images(image_urls))

    print("Starting video downloads...")
    download_videos(20)

    # For audio, if scraped audio URLs are empty, extract using Playwright
    if not audio_urls:
        print("No scraped audio URLs found; extracting using Playwright...")
        audio_urls_extracted = loop.run_until_complete(extract_audio_links())
    else:
        audio_urls_extracted = audio_urls[:20]

    print("Starting audio downloads...")
    loop.run_until_complete(download_audios(audio_urls_extracted))

    print("All downloads complete. Check the 'deepfake_images', 'deepfake_videos', and 'deepfake_audios' directories.")

if __name__ == '__main__':
    main()

Starting image downloads...


RuntimeError: This event loop is already running